## 자료 출처

DHL님의 블로그 - https://thlee33.medium.com/windows-10%EC%97%90%EC%84%9C-geopandas-%EC%84%A4%EC%B9%98%ED%95%98%EA%B8%B0-2021-07-f07efea9294a

## 기본 환경

Anaconda - Python: 3.8.10
윈도우 10, 64bit

In [1]:
import sys
sys.executable

'c:\\Users\\joppu\\anaconda3\\python.exe'

In [2]:
sys.version

'3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]'

## 패키지 다운로드

더 아래의 pip 설치 구문에 있는 패키지별 버전을 참조하여
아래의 사이트에서 각 패키지 설치 파일(.whl)을 다운로드 받습니다.

https://www.lfd.uci.edu/~gohlke/pythonlibs/#shapely  
https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal  
https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona  
https://pypi.org/project/geopandas/#files

## 설치

아나콘다 프롬프트를 관리자 권한으로 실행합니다.  
VSCode로 진행하시는 분은 VSCode 실행을 관리자권한으로 해야 합니다.  
현재 경로를 확인하고 이를 감안하여 적정 경로에 설치 파일을 이동해놓습니다.  
저는 강의자료가 있는 곳에 geopandas를 폴더를 만들고 이 폴더 안에 설치 파일들을 옮겨놓음  

아래의 설치/업데이트 구문을 한줄씩 실행합니다.  
먼저, pip와 numpy를 업그레이드 합니다.  

In [3]:
!python -m pip install — upgrade pip
!pip install — upgrade numpy

ERROR: Invalid requirement: '—'
ERROR: Invalid requirement: '—'


그 다음 아래 패키지를 순차적으로 설치합니다

In [4]:
!pip3.10 install pyproj 
!pip install geopandas/shapely-1.8.2-cp310-cp310-win_amd64.whl
!pip install geopandas/GDAL-3.4.3-cp310-cp310-win_amd64.whl
!pip install geopandas/Fiona-1.8.21-cp310-cp310-win_amd64.whl
!pip install geopandas/geopandas-0.12.2-py3-none-any.whl

'pip3.10'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


Processing c:\users\joppu\onedrive\문서\data minning\suji_blog\posts\geopandas\geopandas\shapely-1.8.2-cp310-cp310-win_amd64.whl
shapely is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing c:\users\joppu\onedrive\문서\data minning\suji_blog\posts\geopandas\geopandas\gdal-3.4.3-cp310-cp310-win_amd64.whl
GDAL is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing c:\users\joppu\onedrive\문서\data minning\suji_blog\posts\geopandas\geopandas\fiona-1.8.21-cp310-cp310-win_amd64.whl
Fiona is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing c:\users\joppu\onedrive\문서\data minning\suji_blog\posts\geopandas\geopandas\geopandas-0.12.2-py3-none-any.whl
geopandas is already installed with the same version as the provided wheel. Use --force-reinstall to f

실행이 잘 되는지 확인

In [5]:
import geopandas as gpd

In [6]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')

In [12]:
!pip install geopy

from geopy.geocoders import Nominatim


     ------------------------------------ 119.8/119.8 kB 333.1 kB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [14]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

29 58m 44.976s N, 31 8m 3.17625s E
هرم خوفو, شارع ابو الهول السياحي, نزلة البطران, الجيزة, 12125, مصر


In [15]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 29.97916
Longitude: 31.134215625236113


In [17]:
universities = pd.read_csv("C:/Users/joppu/OneDrive/문서/data minning/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


- 아래의 코드에서 try함수와 except함수를 사용하는 이유는 코드를 추출하는 과정에서 해당되지않는 값이 에러로 발생을 하지 않게 오류가되는 것들은 넘기고 해당되는 값만 추출하기 위한 함수라고 알고있으면 됨

In [18]:
def my_geocoder(row):
    try:        #이 try와 except를 사용하응 이유 : 코드를 돌리는 과정에서 에러로 인한 멈춤을 없에고 해당되지 않는 것들은 넘기고 해당되는 것들만 추출하기 위함을 알아야한다.
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

91.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.759037,-1.252430,POINT (-1.25243 51.75904)
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.562772,7.580947,POINT (7.58095 47.56277)
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179)


In [19]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

In [20]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [21]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((180.00000 71.51571, 180.00000 ..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [22]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,144373535.0,1699876
1,Norway,5347896.0,403336
2,France,67059887.0,2715518
3,Sweden,10285453.0,530883
4,Belarus,9466856.0,63080


In [23]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((180.00000 71.51571, 180.00000 ...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080


In [24]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
#print(f"We located {} universities.",len(universities))처럼 코드를 사용하는 것 보다 아래처럼 코드를 활용하는 것이 좋다.

print(f"We located {len(universities)} universities.")

#print("Only {} of the universities were located in Europe (in {} different countries).".format(
    #len(european_universities), len(european_universities.name.unique())))

print("Only {len(european_universities)} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

We located 91 universities.
Only 86 of the universities were located in Europe (in 14 different countries).


,Name,Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
0,University of Oxford,51.759037,-1.252430,POINT (-1.25243 51.75904),28,United Kingdom,66834405.0,2829108
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062),28,United Kingdom,66834405.0,2829108
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896),28,United Kingdom,66834405.0,2829108
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179),28,United Kingdom,66834405.0,2829108
5,London School of Economics and Political Science,51.514211,-0.116808,POINT (-0.11681 51.51421),28,United Kingdom,66834405.0,2829108
